# Preface

In this notebook, we investigate some simple examples to illustrate the Probably Approximately Correct (PAC) Framework for studying generalization errors. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_context('notebook', font_scale=1.25, rc={"lines.linewidth": 2.5})
sns.set_style("darkgrid")
np.random.seed(123)  # For reproducibility

# Coin Tossing

Let us start with a coin flipping example. Suppose we want to flip a possibly *biased* coin, which comes up heads (1) with probability $p^*$, and tails (0) with probability $1-p^*$.

How do we flip a coin on a computer? The easiest way is to generate a random variable
$$
    x \sim \mu \equiv \mathcal{U}[0,1]  
    \qquad
    \text{(Uniform Distribution in the unit interval) }
$$
and return 1 (representing heads) if $x\leq p^*$, and 0 otherwise.

We can write this in terms of an indicator function
$$
    f^*(x) = \mathbb{I}_{x\leq p^*}
$$
In other words,
$$
    f^*(x) =
    \begin{cases}
        1 & x_i \leq p^* \\
        0 & x_i > p^*
    \end{cases}
$$

In [ ]:
def indicator(x, p):
    """
    Returns 1 (Heads) if x < p and 0 (Tails) otherwise    
    """
    return np.where(x < p, np.ones_like(x), np.zeros_like(x))

In [ ]:
def flip_coin(num_flips, p):
    """
    Flips a coin via generating uniform random variables x
    Returns x, y=indicator(x, p)
    """
    x = np.random.uniform(size=(num_flips, ))
    return x, indicator(x, p)

## Flipping Experiments

We set $p^*$ to be some fixed value (0.3141593). Let us now flip this "coin" by generating a dataset
$$
    \mathcal{D} =
    \{
        x_i\sim \mathcal{U}[0,1],
        \,
        y_i = f^*(x_i) = \mathbb{I}_{x_i \leq p^*}
    \}_{i=1}^{N}
$$

Observe that if repeat this for many times, then the average proportion of heads will become close to $p^*$. This is a consequence of the **law of large numbers**.

In [ ]:
p_star = 0.3141593
f_star = lambda x: indicator(x, p_star)

In [ ]:
proportion_of_heads = []
num_flips = [int(10**j) for j in np.linspace(1, 8, 20)]
for n in num_flips:
    _, flips = flip_coin(num_flips=n, p=p_star)
    proportion_of_heads.append(np.mean(flips))
coin_data = pd.DataFrame({
    'number of flips': num_flips,
    'proportion of heads': proportion_of_heads,
    r'$p^*$': p_star,
})

In [ ]:
coin_data.plot(
    x='number of flips',
    y=['proportion of heads', r'$p^*$'],
    logx=True,
    style=['-', '--']
)

## Algorithm To Return A Consistent $\hat{f}$

Now let us think of an algorithm that guesses $p^*$ from observing the dataset
$$
    \mathcal{D} =
    \{
        x_i, y_i
    \}_{i=1}^{N}
$$

There are of course many candidates. For example, you can use the mean $\hat{p} = \frac{1}{N}\sum_{i=1}^{N} y_i$. But this has the disadvantage that the training error is not 0, i.e. it is not *consistent* with the dataset. 

Instead, we are going to consider another candidate, namely, we pick $\hat{p}$ to be the largest $x_i$ in our dataset such that $y_i = 1$. In other words
\begin{equation}\label{eq:p_hat}
    \hat{p} =
    \begin{cases}
        \max \{ x_i : y_i = 1 \} & \text{there exists } y_i = 1 \\
        0 & \text{otherwise}
    \end{cases}
\end{equation}

Correspondingly, given the dataset, the above algorithm returns a $\hat{f}$ given by
$$
    \hat{f}(x) = \mathbb{I}_{x\leq \hat{p}}
$$
with $\hat{p}$ given by \eqref{eq:p_hat}.

Therefore, in this case our algorithm $\mathcal{A}$ generates a $\hat{f}$ from a dataset $\mathcal{D}$. Symbolically, we can write
$$
    \mathcal{A}(\mathcal{D}) = \hat{f}
$$

In [ ]:
def algorithm_p_hat(x, y):
    y_heads = y==1.0
    if np.all(~y_heads):
        return 0.0
    else:
        return np.max(x[y_heads])

Let us now do some flips and check what $\hat{p}$ and $\hat{f}$ looks like.

In [ ]:
N = 10  # sample size

In [ ]:
x_train, y_train = flip_coin(num_flips=N, p=p_star)
p_hat = algorithm_p_hat(x_train, y_train)
f_hat = lambda x: indicator(x, p_hat)

x_grid = np.linspace(0, 1, 1000)

# Training data
plt.scatter(x_train[y_train==1], y_train[y_train==1], label=r'$y_i = 1$')
plt.scatter(x_train[y_train==0], y_train[y_train==0], label=r'$y_i = 0$')

# f*
plt.plot(x_grid, f_star(x_grid), '--', label=r'$f^*$', c='k', lw=1)
plt.fill_between(x_grid, f_star(x_grid), color='k', alpha=0.1)

# f_hat
plt.plot(x_grid, f_hat(x_grid), ':', label=r'$\hat{f}$', c='b', lw=1)
plt.fill_between(x_grid, f_hat(x_grid), color='b', alpha=0.1)

# Labels
plt.text(p_star, 0.25, r'$x=p^*$', color='k')
plt.text(p_hat, 0.75, r'$x=\hat{p}$', color='b')

plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.legend()

**Important Takeaway**

Notice the following:
1. $\hat{p}$ and thus $\hat{f}$ depends on the dataset. If we resample the dataset, they change. In this sense, they are *random variables*
2. Despite the randomness, due to our construction, the empirical risk $R_\mathrm{emp}(\hat{f})$ is always 0! In other words, our algorithm always returns a *consistent* hypothesis. Moreover, we always have $\hat{p} \leq {p}^*$!
3. When sample size $N$ increases, $\hat{p}$ approaches $p^*$. Equivalently, $\hat{f}$ approaches $f^*$

## The Distribution of $R_\mathrm{pop}(\hat{f})$

Now, let us consider the population risk $R_\mathrm{pop}(\hat{f})$. Unlike the empirical risk, this is not zero.

Looking at the above figures, it is clear that $f^*$ and $\hat{f}$ differ only in the range $\hat{p} < x \leq p^*$. The population risk is given by
\begin{equation}
    R_\mathrm{pop}(\hat{f}) 
    = \mathbb{E}_{x\sim\mu} [\mathbb{I}_{f^*(x)\neq \hat{f}(x)}]
    = \mathbb{P}_{x\sim\mu} [\hat{p} < x \leq p^*]
    = p^* - \hat{p}.
\end{equation}
Again, with each resampling of the training data, $R_\mathrm{pop}$ is going to be random!

Let us now investigate its distribution

In [ ]:
def R_pop(p_hat, p_star):
    """
    Compute R_pop = p_star - p_hat
    """
    return p_star - p_hat

In [ ]:
def sample_p_hats(num_trials, N):
    """
    Generate p_hats by sampling and training
    on datasets of size N
    """
    p_hats = []
    for _ in range(num_trials):
        x_train, y_train = flip_coin(num_flips=N, p=p_star)
        p_hats.append(algorithm_p_hat(x_train, y_train))
    return p_hats

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 4))

for N, a in zip([5, 10, 25], ax):
    p_hats = sample_p_hats(num_trials=10000, N=N)
    R_pops = [R_pop(p, p_star) for p in p_hats]
    
    sns.distplot(
        R_pops,
        kde=False,
        norm_hist=True,
        bins=50,
        ax=a,
    )
    a.set_title(rf'$N = {N}$ (Worst: {max(R_pops):.2f})')
    a.set_xlabel(r'$R_{\mathrm{pop}}$')
    a.set_ylabel('Normalized Frequency')
fig.tight_layout()

Observe that even when sample size increases, the worst case $R_\mathrm{pop}(\hat{f})$ is bad. It makes better sense to not talk about worse case scenarios. The PAC framework is an example of this: it does not care about worse cases, as long as their probability to occur decays as $N$ increases.

More precisely, we want to show that for any error tolerance $\epsilon$ and worst-case tolerance $\delta$, we have
$$
    \mathbb{P}_{\mathcal{D}\sim\mu}
    [
        R_\mathrm{pop}(\hat{f}) \leq \epsilon
    ] \geq 1 - \delta
$$
for $N$ *sufficiently* large.

Let us solve this problem exactly to get some ideas.

---

## The Distribution of $\hat{p}$

Let us now compute the distribution of $\hat{p}$, and thus $R_\mathrm{pop}(\hat{f})$.

Let us compute the cumulative distribution function
\begin{align*}
    &\mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} \leq z] \\
    =& \mathbb{P}_{\mathcal{D}\sim\mu}[x_i \leq z \text{ or } x_i > p^* \text{ for all } i] \\
    =& (\mathbb{P}_{x_i\sim\mu}[x_i \leq z \text{ or } x_i > p^*])^{N}
    \qquad \text{by i.i.d. assumption} \\
    =& (1 - p^* + z)^{N}
\end{align*}

**Check Our Results**

In [ ]:
def p_hat_CDF(z, p_star, N):
    """
    Exact CDF P[p_hat <= z]
    """
    return (1 - p_star + z)**N

In [ ]:
x_grid = np.linspace(0, p_star, 100)

fig, ax = plt.subplots(1, 3, figsize=(15, 4))

for N, a in zip([5, 15, 30], ax):
    p_hats = sample_p_hats(num_trials=10000, N=N)
    sns.distplot(p_hats,
                 kde=False,
                 norm_hist=True,
                 bins=50,
                 hist_kws=dict(cumulative=True),
                 label='Empirical CDF',
                 ax=a)
    a.plot(x_grid, p_hat_CDF(x_grid, p_star, N=N), label='Exact CDF')
    a.set_title(rf'$N = {N}$')
    a.set_xlabel(r'$x$')
    a.set_ylabel(r'$\mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} \leq x]$')
    #     a.set_ylabel(r'$\mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop}(\hat{f}) \leq x]$')
    a.legend()
fig.tight_layout()

## Distribution of $R_\text{pop}(\hat{f})$

From the distribution of $\hat{p}$, we can directly get the distribution of $R_{pop} = p^* - \hat{p}$.

We have
\begin{align*}
    &\mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop} \leq z] \\
    =&\mathbb{P}_{\mathcal{D}\sim\mu}[p^* - \hat{p} \leq z] \\
    =&\mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} \geq p^* - z] \\
    =& 1 - \mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} < p^* - z] \\
    =& 1 - \mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} \leq p^* - z] 
    + \mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} = p^* - z]
\end{align*}

From our previous result, we know that 
$$
    \mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} \leq p^* - z] = (1-p^*+[p^* - z])^N = (1-z)^N.
$$
But we have to be a bit careful about $\mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} = p^* - z]$. From the CDF, we know that as long as $z \neq p^*$, this is going to be zero. If $z = p^*$, then we have $\mathbb{P}_{\mathcal{D}\sim\mu}[\hat{p} = 0] = (1-p^*)^N$.

Hence, we have
\begin{equation}
    \mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop} \leq z] =
    \begin{cases}
        1 - (1-z)^N & z < p^* \\
        1 & z = p^*
    \end{cases}
\end{equation}

**Check Our Result**

In [ ]:
def R_pop_CDF(z, p_star, N):
    """
    Exact CDF P[p_hat <= z]
    """
    return np.where(z==p_star, np.ones_like(z), 1 - (1-z)**N)

In [ ]:
x_grid = np.linspace(0, p_star, 1000)

fig, ax = plt.subplots(1, 3, figsize=(15, 4))

for N, a in zip([5, 15, 30], ax):
    p_hats = sample_p_hats(num_trials=10000, N=N)
    R_pops = [p_star-p for p in p_hats]
    sns.distplot(R_pops,
                 kde=False,
                 norm_hist=True,
                 bins=50,
                 hist_kws=dict(cumulative=True),
                 label='Empirical CDF',
                 ax=a)
    a.plot(x_grid, R_pop_CDF(x_grid, p_star, N=N), label='Exact CDF')
    a.set_title(rf'$N = {N}$')
    a.set_xlabel(r'$x$')
    a.set_ylabel(r'$\mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop}(\hat{f}) \leq x]$')
    a.legend()
fig.tight_layout()

## The PAC Result

In this case, we actually have a simple PAC result. Let us assume that $\epsilon < p^*$ for simplicity, then, we have
$$
    \mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop} \leq \epsilon] = 1 - (1-\epsilon)^N
$$

Thus, we obtain a PAC result if $N$ is large enough so that $(1-\epsilon)^N \leq \delta$, in which case we would have
$$
    \mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop} \leq \epsilon] \geq 1 - \delta
$$

Solving for $(1-\epsilon)^N \leq \delta $ we get
$$
    N \geq \log \frac{1}{\delta} / \log \frac{1}{1-\epsilon}  \geq \frac{1}{\epsilon} \log \frac{1}{\delta}
$$
In the last line we used the fact that $\log(z) \leq 1 + z$ so $-\log(1/z) \geq -1 - 1/z$, but this is not terribly important.

## Empirical Test Error Prediction

There are other ways to write the previous result. Let us take $N = \frac{1}{\epsilon} \log \frac{1}{\delta}$. Then, we have $\epsilon = \frac{1}{N} \log \frac{1}{\delta}$.

Then, the previous PAC result says that
$$
    \mathbb{P}_{\mathcal{D}\sim\mu}
    \left[
        R_\mathrm{pop} \leq \frac{1}{N} \log \frac{1}{\delta}
    \right]
    \geq 1 - \delta
$$
which says that with high probability, $R_\mathrm{pop}$ is going to be of order $\mathcal{O}(1/N)$! 

In this special case, we can actually compute the expectation of $R_\mathrm{pop}$ as follows:
\begin{align*}
    &\mathbb{E}_{\mathcal{D}\sim\mu}[R_\mathrm{pop}(\hat{f})] \\
    =& \int_{0}^{p^*}
    \mathbb{P}_{\mathcal{D}\sim\mu}[R_\mathrm{pop}(\hat{f}) > z] dz \\
    =& \int_{0}^{p^*} (1 - z)^N dz \\
    =& \frac{1 - (1-p^*)^{N+1}}{N+1} \\
    =& \mathcal{O}\left(\frac{1}{N}\right)
\end{align*}
Therefore, the expected population risk decreases like $1/N$ as the sample size $N$ increases!

**Check Our Result**

In [ ]:
num_repeats = 20

test_results = []
N_grid = [5, 10, 50, 100, 500, 1000]
for N in N_grid:
    for _ in range(num_repeats):
        x_train, y_train = flip_coin(num_flips=N, p=p_star)
        x_test, y_test = flip_coin(num_flips=10000, p=p_star)
        p_hat = algorithm_p_hat(x_train, y_train)
        f_hat = lambda x: indicator(x, p_hat)
        test_results.append([N, np.mean(f_hat(x_test) != y_test)])

In [ ]:
test_results = pd.DataFrame(test_results, columns=[r'$N$', 'Test Error'])

In [ ]:
ax = sns.lineplot(
    x=r'$N$',
    y='Test Error',
    data=test_results,
    label=r'Test Error vs $N$'
)
ax.set(xscale='log', yscale='log')
ax.loglog(N_grid, [1/N for N in N_grid], '--', label=r'$1/N$')
ax.legend()

# Universal Generalization Bound for Finite Hypothesis Space

Now let us look at why we should expect some universal generalization bounds for finite hypothesis space.

First, we are going to define a class of arbitrary binary classifiers based on a Gaussian RBF function. The class below implements this.

Out of this finite set of RBF classifiers, we are going to pick one to be our oracle $f^*$.

In [ ]:
class RandomRBFHypothesisSpace(object):
    def __init__(self, num_h):
        self.num_h = num_h
        self.mean = np.random.uniform(size=(num_h, 2))
        self.cov = np.random.uniform(size=(num_h, 2, 2))
        self.cov_inv = np.asarray([
            np.linalg.inv(c.T @ c + 0.1 * np.identity(c.shape[0]))
            for c in self.cov
        ])
        self.j_star = np.random.choice(range(num_h))

    def hxj(self, x, j):
        norm = np.sum(
            (x - self.mean[j]) @ self.cov_inv[j] * (x - self.mean[j]), axis=1)
        pdf = np.exp(-norm)
        return (np.sign(pdf - 0.5) + 1) / 2

    def f_star(self, x):
        return self.hxj(x, self.j_star)

    def h(self, j):
        return lambda x: self.hxj(x, j)

In [ ]:
def plot_heat_map(h, ax):
    x1_mesh, x2_mesh = np.mgrid[0:1:100j, 0:1:100j]
    positions = np.vstack([x1_mesh.ravel(), x2_mesh.ravel()]).T
    values = h(positions)
    ax.contourf(np.linspace(0, 1, 100), np.linspace(0, 1, 100), values.reshape(100, 100), cmap='GnBu')

In [ ]:
inds = RandomRBFHypothesisSpace(num_h=1000)

Let us plot the optimal $f^* \in \mathcal{H}$

In [ ]:
plot_heat_map(inds.f_star, plt.axes())

Now let us plot the other $h \in \mathcal{H}$

In [ ]:
from math import ceil

num_plots = 15
num_plots = min(num_plots, inds.num_h)

num_cols = 5
num_rows = ceil(num_plots / num_cols)

fig, ax = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, 4*num_rows))

for a in ax.ravel():
    j = np.random.choice(range(inds.num_h))
    plot_heat_map(inds.h(j), ax=a)

## Consistency vs Increasing Data Count

First, let us sample $N$ data points from $f^*$ according to the uniform distribution in the unit square. However, note that this would work with *any* distribution (Try it yourself as an exercise!)

Suppose our algorithm returns a consistent hypothesis, meaning that it must have zero training error. What happens when you increase the number of data points?

Let us investigate!

We randomly draw $N$ samples, uniformly in the unit square. This builds the dataset
$$
    \mathcal{D} = \{ x_i \sim \mu\equiv\mathcal{U}[0,1]^2 , y_i = f^*(x_i) \}_{i=1}^{N}
$$

In [ ]:
N = 5

x_train = np.random.uniform(size=(N, 2))
y_train = inds.f_star(x_train)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
plot_heat_map(inds.f_star, ax=ax)
positives = y_train == 1.0
ax.scatter(x_train[:, 1][positives], x_train[:, 0][positives], c='lightblue', label='1')
ax.scatter(x_train[:, 1][~positives], x_train[:, 0][~positives], c='green', label='0')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.legend(bbox_to_anchor=(1.1, 1.05))

Now, let us plot all the $h\in\mathcal{H}$ which are consistent with the dataset

In [ ]:
def consistency_check(h, x, y):
    y_pred = h(x)
    return np.all(y_pred == y)

In [ ]:
num_plots = 15

check_results = [consistency_check(inds.h(j), x_train, y_train) for j in range(inds.num_h)]
consistent_subset = np.arange(inds.num_h)[check_results]
print(f'Number of consistent hypotheses: {len(consistent_subset)}')

num_plots = min(num_plots, len(consistent_subset))

num_cols = 5
num_rows = ceil(num_plots / num_cols)
num_cols = min(num_cols, len(consistent_subset))

fig, ax = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, 4*num_rows), squeeze=False)

for a in ax.ravel():
    j = np.random.choice(consistent_subset)
    plot_heat_map(inds.h(j), ax=a)
    a.scatter(x_train[:, 1][positives], x_train[:, 0][positives], c='lightblue', label='1')
    a.scatter(x_train[:, 1][~positives], x_train[:, 0][~positives], c='green', label='0')

What happpens when we increase $N$?